### Dependencies

In [9]:
import pandas as pd
import matplotlib.pyplot as plt

from time import strftime
from time import time

import warnings
warnings.filterwarnings("ignore")

### Needed Functions

In [2]:
def sheetsTTC(xlsx_file):
    
    """ 
    Reads in an Excel file and concatenates each sheet into 
    a returned pandas DataFrame.    
    
    Note:
        - Requires: pandas
        - Requires: Each worksheet must have exact matching
                    n columns, column order, column labels
                    and column dtypes.
    """
    
    xls = pd.ExcelFile(xlsx_file)
    
    out_df = pd.DataFrame()
    
    for sheet in xls.sheet_names:
        
        df = pd.read_excel(xls, sheet_name = sheet)
        
        df = df[[
            'Report Date', 'Route', 'Time', 'Day', 'Location', 'Incident', 'Min Delay', 'Min Gap', 'Direction', 'Vehicle'
        ]]
        
        out_df = pd.concat([out_df, df])
        
    return out_df

def excelDate(excel_time):
    
    """
    Converts excel datetime float format to pandas datetime
    
    """
    
    return pd.to_datetime('1900-01-01') + pd.to_timedelta(excel_time, 'D')


### Buses

In [3]:
# Bus 2019
bus2019_xls = pd.ExcelFile('Bus_2019.xlsx')

jan = pd.read_excel(bus2019_xls, 'Jan 2019')
feb = pd.read_excel(bus2019_xls, 'Feb 2019')
mar = pd.read_excel(bus2019_xls, 'Mar 2019')
apr = pd.read_excel(bus2019_xls, 'Apr 2019')
may = pd.read_excel(bus2019_xls, 'May 2019')

# April is the only month in the 5 1/2 years of data
# with the column 'Incident ID', so it gets dropped
apr = apr.drop(columns={'Incident ID'})

# Rename the two mis-labelled columns
apr = apr.rename(columns={'Delay': 'Min Delay', 'Gap': 'Min Gap'})

bus19 = pd.concat([jan, feb])
bus19 = pd.concat([bus19, mar])
bus19 = pd.concat([bus19, apr])
bus19 = pd.concat([bus19, may])

# Apply sheetsTTC Function
bus18 = sheetsTTC('Bus_2018.xlsx')
bus17 = sheetsTTC('Bus_2017.xlsx')
bus16 = sheetsTTC('Bus_2016.xlsx')
bus15 = sheetsTTC('Bus_2015.xlsx')
bus14 = sheetsTTC('Bus_2014.xlsx')

# Add Year Columns
bus19['Year'] = 2019
bus18['Year'] = 2018
bus17['Year'] = 2017
bus16['Year'] = 2016
bus15['Year'] = 2015
bus14['Year'] = 2014

# Combine 2014 - 2019
buses = pd.concat([bus19, bus18])
buses = pd.concat([buses, bus17])
buses = pd.concat([buses, bus16])
buses = pd.concat([buses, bus15])
buses = pd.concat([buses, bus14])

# Add Bus labeled Type column
buses['Type'] = 'Bus'

In [4]:
buses.head()

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,Year,Type
0,43466,39,0.009028,Tuesday,NECR,Mechanical,9.0,18.0,W/B,1794.0,2019,Bus
1,43466,111,0.010417,Tuesday,Eglington,Mechanical,15.0,30.0,S/B,8065.0,2019,Bus
2,43466,35,0.012500,Tuesday,Finch,Mechanical,9.0,18.0,S/B,3275.0,2019,Bus
3,43466,25,0.020833,Tuesday,Don Mills Rd/Eglinton Ave E,Mechanical,9.0,18.0,N/B,8840.0,2019,Bus
4,43466,36,0.027778,Tuesday,Humberwood,Investigation,9.0,18.0,E/B,9119.0,2019,Bus


### Streetcar

In [5]:
# Apply sheetsTTC Function
streetcar19 = sheetsTTC('Streetcar_2019.xlsx')
streetcar18 = sheetsTTC('Streetcar_2018.xlsx')
streetcar17 = sheetsTTC('Streetcar_2017.xlsx')
streetcar16 = sheetsTTC('Streetcar_2016.xlsx')
streetcar15 = sheetsTTC('Streetcar_2015.xlsx')
streetcar14 = sheetsTTC('Streetcar_2014.xlsx')

# Add Year Columns
streetcar19['Year'] = 2019
streetcar18['Year'] = 2018
streetcar17['Year'] = 2017
streetcar16['Year'] = 2016
streetcar15['Year'] = 2015
streetcar14['Year'] = 2014

# Combine 2014 - 2019
streetcars = pd.concat([streetcar19, streetcar18])
streetcars = pd.concat([streetcars, streetcar17])
streetcars = pd.concat([streetcars,streetcar16])
streetcars = pd.concat([streetcars, streetcar15])
streetcars = pd.concat([streetcars, streetcar14])

# Add Streetcar labeled Type column
streetcars['Type'] = 'Streetcar'

In [6]:
streetcars.head()

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,Year,Type
0,43466,301,0.047222,Tuesday,Queen/Braodview,Held By,6.0,13.0,E/B,4193.0,2019,Streetcar
1,43466,511,0.062500,Tuesday,Bathurst/College,Investigation,5.0,10.0,N/B,1038.0,2019,Streetcar
2,43466,306,0.069444,Tuesday,Dundas West stn.,Mechanical,8.0,16.0,W/B,4146.0,2019,Streetcar
3,43466,505,0.131944,Tuesday,Lansdowne and Dundas,Mechanical,6.0,12.0,E/B,8416.0,2019,Streetcar
4,43466,310,0.163194,Tuesday,Spadina and Lakshore,Held By,20.0,30.0,N/B,4465.0,2019,Streetcar


### Surface Routes (Streetcar + Buses)

In [17]:
surface = pd.concat([buses, streetcars])

In [18]:
surface['Report Date'] = excelDate(surface['Report Date'])
surface['Time'] = round(((surface['Time'] * 86400)/60)/60, 2)

In [20]:
surface.to_csv('ttc_surface_route_delays.csv')
surface.sample(25)

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,Year,Type
1732,2014-06-08,47,18.67,Friday,Queen&McDonell,Investigation,17.0,34.0,N,1018.0,2014,Bus
128,2014-03-03,196,15.28,Saturday,Downsview Stn,Utilized Off Route,15.0,30.0,SB,1377.0,2014,Bus
1025,2017-08-08,61,9.75,Sunday,Eglinton Stn,Mechanical,20.0,40.0,n,1702.0,2017,Bus
6155,2015-10-02,35,13.12,Wednesday,Alliance,Investigation,18.0,36.0,N,1327.0,2015,Bus
657,2016-08-21,506,23.52,Friday,Gerrard at Carlaw,Held By,9.0,17.0,E/B,4126.0,2016,Streetcar
4507,2018-11-24,144,18.18,Thursday,St. Dennis/Deauville,Diversion,1.0,1.0,N/B,8696.0,2018,Bus
407,2018-07-12,504,7.63,Tuesday,Queen and Cannought,Late Leaving Garage,9.0,18.0,W/B,4463.0,2018,Streetcar
4623,2016-11-22,45,22.07,Sunday,Kipling and John Garland,Mechanical,10.0,20.0,n,7959.0,2016,Bus
2168,2017-05-14,39,5.37,Friday,BDFI,Late Leaving Garage,5.0,10.0,w,1708.0,2017,Bus
563,2019-05-24,506,12.92,Wednesday,High Park Loop,Emergency Services,5.0,10.0,W/B,4199.0,2019,Streetcar


### Quick Look

In [10]:
surface.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 494169 entries, 0 to 873
Data columns (total 12 columns):
Report Date    494169 non-null datetime64[ns]
Route          494169 non-null int64
Time           494169 non-null float64
Day            494169 non-null object
Location       493238 non-null object
Incident       493238 non-null object
Min Delay      493855 non-null float64
Min Gap        493392 non-null float64
Direction      484880 non-null object
Vehicle        428708 non-null float64
Year           494169 non-null int64
Type           494169 non-null object
dtypes: datetime64[ns](1), float64(4), int64(2), object(5)
memory usage: 49.0+ MB


In [11]:
surface.describe()

,Route,Time,Min Delay,Min Gap,Vehicle,Year
count,494169.000000,4.941690e+05,493855.000000,493392.000000,428708.000000,494169.000000
mean,167.907617,6.596399e+01,18.715536,27.504749,5410.035042,2016.142395
std,248.292435,7.378415e+03,356.612916,67.871773,3123.756859,1.597574
min,1.000000,0.000000e+00,-54.000000,-2.000000,0.000000,2014.000000
25%,42.000000,7.300000e+00,6.000000,11.000000,1669.000000,2015.000000
50%,87.000000,1.275000e+01,10.000000,18.000000,7428.000000,2016.000000
75%,191.000000,1.617000e+01,15.000000,28.000000,8145.000000,2018.000000
max,106117.000000,1.046256e+06,246245.000000,6528.000000,163242.000000,2019.000000


In [12]:
surface['Incident'].value_counts()

Mechanical                          192907
Late Leaving Garage                  72983
Utilized Off Route                   70781
General Delay                        64670
Investigation                        48086
Diversion                            24870
Emergency Services                   10336
Held By                               8321
Overhead - Pantograph                  142
Vision                                  61
Late Leaving Garage - Mechanical        48
Late Leaving Garage - Operator          25
Late Leaving Garage - Management         8
Name: Incident, dtype: int64

In [14]:
surface['Type'].value_counts()

Bus          421625
Streetcar     72544
Name: Type, dtype: int64

In [15]:
surface['Day'].value_counts()

Thursday     82791
Tuesday      82461
Wednesday    81684
Friday       80781
Monday       74617
Saturday     49662
Sunday       42173
Name: Day, dtype: int64